In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(mltools)
library(data.table)
library(dplyr)
library(ggplot2)
library(ggiraph)
library('mclust')

IN_DIR <- '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcga/output/'
OUT_DIR <- '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcga/output_ARI/'

gnn_data <- read.csv(paste0(IN_DIR, 'gnn_predictions.tsv'), sep = "\t", row.names = NULL)
resnet_data <- read.csv(paste0(IN_DIR, 'resnet_predictions.tsv'), sep = "\t", row.names = NULL)

data.table 1.16.0 using 8 threads (see ?getDTthreads).  Latest news: r-datatable.com

Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

                   __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.


In [3]:
%%R
str(gnn_data)
str(resnet_data)

'data.frame':	722 obs. of  17 variables:
 $ sample_ids         : chr  "SRR1810239" "SRR7161462" "SRR2033443" "SRR6680502" ...
 $ tissues            : chr  "Liver" "Intestine" "Liver" "Liver" ...
 $ target             : int  6 4 6 6 3 4 1 6 6 7 ...
 $ prediction         : int  6 4 0 6 5 4 1 6 6 7 ...
 $ confidence_class_0 : num  6.75e-04 5.54e-08 9.96e-01 5.84e-05 2.18e-01 ...
 $ confidence_class_1 : num  1.12e-02 3.46e-03 3.47e-05 2.21e-06 9.94e-05 ...
 $ confidence_class_2 : num  4.52e-05 1.70e-03 5.17e-06 2.83e-06 3.61e-05 ...
 $ confidence_class_3 : num  1.93e-07 2.36e-07 4.77e-04 4.09e-08 6.64e-03 ...
 $ confidence_class_4 : num  0.004628 0.993361 0.000561 0.002649 0.071297 ...
 $ confidence_class_5 : num  3.07e-03 4.33e-06 7.01e-04 1.03e-03 6.62e-01 ...
 $ confidence_class_6 : num  7.70e-01 5.63e-04 4.07e-06 9.95e-01 1.35e-02 ...
 $ confidence_class_7 : num  1.86e-01 1.89e-11 2.21e-04 1.47e-04 2.35e-02 ...
 $ confidence_class_8 : num  3.32e-10 1.74e-10 5.15e-08 1.03e-08 1.41e-04 .

In [4]:
%%R
print(unique(gnn_data$issues))
print(gnn_data)

NULL
     sample_ids   tissues target prediction confidence_class_0
1    SRR1810239     Liver      6          6       6.752172e-04
2    SRR7161462 Intestine      4          4       5.535472e-08
3    SRR2033443     Liver      6          0       9.961665e-01
4    SRR6680502     Liver      6          6       5.835105e-05
5    SRR5009821     Heart      3          5       2.183444e-01
6    SRR7161319 Intestine      4          4       2.879065e-07
7   SRR10399918     Brain      1          1       4.853998e-21
8    SRR1796662     Liver      6          6       7.517284e-05
9    SRR2061987     Liver      6          6       7.151351e-04
10   SRR5273652      Lung      7          7       1.250521e-02
11   SRR6194324     Liver      6          6       7.770097e-05
12   SRR7511612      Lung      7          7       1.829190e-03
13   SRR9029889     Liver      6          6       2.404327e-05
14   SRR7641150     Liver      6          6       4.319123e-05
15   SRR3161663    Kidney      5          5       

259  SRR5140170   Adipose      0          0       4.651552e-01
260  SRR6253451     Heart      3          3       6.361237e-03
261  SRR1002639      Lung      7          7       4.195549e-04
262  SRR9591111     Brain      1          1       3.981479e-05
263  ERR2588637    Testes     12         12       1.532512e-10
264 SRR10399957     Brain      1          1       7.044388e-14
265  SRR7161299 Intestine      4          4       3.955832e-07
266  SRR9029924     Liver      6          6       1.749085e-04
267  SRR6680458     Liver      6          6       7.845017e-05
268  SRR3322942     Liver      6          5       1.273379e-01
269  SRR5952080    Muscle      9          9       5.383883e-04
270  SRR9956116       Eye      2          2       9.250442e-05
271  SRR7161460 Intestine      4          4       2.400116e-07
272  SRR8885438   Adipose      0          4       1.329740e-04
273 SRR10321917     Brain      1          1       2.776898e-04
274  SRR5277041     Brain      1          1       3.215

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



      4          6       4.103256e-06
668  SRR3197176     Liver      6          6       1.365230e-05
669  SRR5517156     Liver      6          6       2.241946e-04
670   SRR945382    Kidney      5          5       1.248423e-03
671  ERR2654324    Testes     12         12       2.311669e-07
672  SRR3182845  Pancreas     10         10       1.447099e-01
673  SRR5758674    Muscle      9          9       1.268514e-04
674 SRR10350444     Heart      3          3       2.173124e-02
675  SRR5275512     Liver      6          6       7.501382e-06
676  SRR5127007   Adipose      0          4       2.756188e-01
677   DRR060863 Intestine      4          4       2.168269e-03
678  SRR9591138     Brain      1          1       1.440756e-04
679  SRR1619423     Brain      1          1       1.963207e-05
680  SRR8474311 Intestine      4          4       1.195682e-01
681  SRR3156189     Liver      6          6       3.641692e-05
682  SRR3714615    Kidney      5          5       1.152797e-01
683  SRR8285549  

160       1.171699e-05       1.000582e-04       4.774316e-06       2.694915e-02
161       4.516849e-06       1.408844e-05       2.789386e-07       2.617635e-02
162       1.653752e-01       4.899231e-05       6.048513e-09       8.131159e-01
163       5.554841e-04       2.099515e-04       1.099002e-04       6.324012e-01
164       9.827811e-01       1.602843e-02       2.342208e-05       2.699841e-04
165       7.967366e-01       1.743014e-01       6.986748e-06       1.765178e-03
166       8.876507e-01       1.119284e-01       4.240950e-05       9.235503e-05
167       9.645473e-01       3.495654e-02       6.282021e-05       1.000881e-05
168       3.747739e-01       1.264984e-03       2.669954e-02       2.130110e-04
169       3.151157e-07       9.390886e-06       6.541355e-08       3.426295e-03
170       1.016172e-06       3.961429e-04       1.371738e-06       3.473400e-03
171       1.826755e-05       6.444561e-05       8.383712e-06       1.053967e-02
172       1.285675e-03       1.666718e-0

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        4.035113e-04
287       6.821531e-04        9.606911e-02        3.491478e-06
288       2.812723e-04        5.346509e-08        1.074287e-03
289       7.623856e-04        1.141507e-07        5.088026e-05
290       2.193537e-04        2.467097e-04        8.049683e-02
291       1.497386e-03        3.681092e-10        8.365672e-05
292       1.133821e-11        8.277640e-11        9.651183e-11
293       2.540631e-02        1.065174e-02        3.112949e-02
294       5.372370e-02        1.915307e-04        5.733934e-03
295       2.998838e-02        2.873762e-04        9.477530e-04
296       1.192588e-04        4.379041e-09        1.718219e-04
297       1.372275e-10        1.024162e-11        9.827345e-09
298       6.541264e-04        8.626184e-07        3.707276e-04
299       5.546214e-03        1.391470e-09        7.929453e-01
300       2.168842e-04        2.809534e-08        1.205534e-04
301       4.105826e-02        3.065779e-04        2.705281e-03
302       6.042689e-05        3.57

546       7.146780e-04        6.003816e-08        4.061547e-05
547       7.042671e-04        1.428782e-08        1.122554e-09
548       1.025575e-04        1.733813e-08        4.151087e-05
549       5.627185e-05        3.577474e-01        7.523680e-11
550       1.659980e-03        1.229123e-05        2.509560e-04
551       4.760945e-04        8.035387e-01        1.804011e-06
552       6.305396e-04        4.175419e-04        2.659394e-05
553       8.927102e-05        1.644848e-08        3.439455e-04
554       6.897165e-05        4.289913e-05        3.699640e-03
555       2.917672e-04        2.343688e-08        9.750866e-05
556       1.099916e-02        9.109583e-04        6.661524e-03
557       9.504269e-04        4.690612e-10        1.657605e-04
558       7.530882e-01        5.153884e-03        6.365125e-02
559       8.195442e-01        2.621053e-04        4.048720e-04
560       8.426525e-06        3.457521e-09        5.232829e-05
561       9.887142e-11        5.279893e-08        4.035

218        2.950988e-03
219        7.103852e-07
220        3.954563e-04
221        2.825742e-04
222        1.301063e-02
223        1.859936e-05
224        1.650830e-04
225        1.402984e-05
226        6.299113e-04
227        7.436060e-04
228        1.759239e-05
229        5.764531e-06
230        2.254665e-02
231        1.560417e-03
232        1.005589e-05
233        2.847089e-04
234        3.238812e-12
235        2.519788e-05
236        2.305293e-03
237        1.135673e-04
238        4.578183e-03
239        1.128439e-06
240        1.005917e-05
241        3.080257e-04
242        1.209110e-04
243        6.310239e-05
244        1.157779e-05
245        1.623097e-05
246        2.759846e-04
247        1.046605e-04
248        1.616908e-03
249        2.446180e-05
250        4.842615e-02
251        4.078568e-03
252        1.989788e-03
253        1.910222e-05
254        5.734080e-03
255        1.895287e-05
256        1.583891e-04
257        3.857172e-07
258        5.493703e-07
259        4.153

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
%%R
gnn_ari <- adjustedRandIndex(gnn_data$target, gnn_data$prediction)
resnet_ari <- adjustedRandIndex(resnet_data$target, resnet_data$prediction)
print(paste0("gnn_ari: ", gnn_ari))
print(paste0("resnet_ari: ", resnet_ari))

[1] "gnn_ari: 0.790068183588844"
[1] "resnet_ari: 0.859955565585395"


In [ ]:
%%R
unique_tissues <- unique(c(gnn_data$target, gnn_data$prediction))
names <- unique(gnn_data$tissues)
gnn_results <- data.frame(names = names, tissue = unique_tissues)

gnn_results$TP <- 0
gnn_results$TN <- 0
gnn_results$FP <- 0
gnn_results$FN <- 0

for (tissue in unique_tissues) {
  for (i in 1:nrow(gnn_data)) {
    if (gnn_data$target[i] == tissue) {
      if (gnn_data$prediction[i] == tissue) {
        gnn_results[gnn_results$tissue == tissue, "TP"] <- gnn_results[gnn_results$tissue == tissue, "TP"] + 1
      } else {
        gnn_results[gnn_results$tissue == tissue, "FN"] <- gnn_results[gnn_results$tissue == tissue, "FN"] + 1
      }
    } else {
      if (gnn_data$prediction[i] == tissue) {
        gnn_results[gnn_results$tissue == tissue, "FP"] <- gnn_results[gnn_results$tissue == tissue, "FP"] + 1
      } else {
        gnn_results[gnn_results$tissue == tissue, "TN"] <- gnn_results[gnn_results$tissue == tissue, "TN"] + 1
      }
    }
  }
}

gnn_results$POSITIVES <- gnn_results$TP + gnn_results$FN

gnn_results$MCC <- (gnn_results$TP * gnn_results$TN - gnn_results$FP * gnn_results$FN) / 
  sqrt((gnn_results$TP + gnn_results$FP) * (gnn_results$TP + gnn_results$FN) * (gnn_results$TN + gnn_results$FP) * (gnn_results$TN + gnn_results$FN))

gnn_results$TPR <- gnn_results$TP / (gnn_results$TP + gnn_results$FN)

gnn_results$FPR <- gnn_results$FP / (gnn_results$FP + gnn_results$TN)

gnn_results$TNR <- gnn_results$TN / (gnn_results$TN + gnn_results$FP)

gnn_results$FNR <- gnn_results$FN / (gnn_results$FN + gnn_results$TP)

gnn_results$Precision <- gnn_results$TP / (gnn_results$TP + gnn_results$FP)

gnn_results$Recall <- gnn_results$TP / (gnn_results$TP + gnn_results$FN)

gnn_results$F1_score <- 2 * (gnn_results$Precision * gnn_results$Recall)/(gnn_results$Precision + gnn_results$Recall)

print(gnn_results)



write.table(gnn_results, file = paste(OUT_DIR, "gnn_MCC_Result.txt"), sep="\t", row.names = FALSE, quote = FALSE)

       names tissue  TP  TN FP FN POSITIVES       MCC       TPR         FPR
1      Liver      6 199 495 10 18       217 0.9071029 0.9170507 0.019801980
2  Intestine      4  54 652  6 10        64 0.8593955 0.8437500 0.009118541
3      Heart      3  12 691 11  8        20 0.5460590 0.6000000 0.015669516
4      Brain      1 161 526 18 17       178 0.8697684 0.9044944 0.033088235
5       Lung      7  35 682  3  2        37 0.9297737 0.9459459 0.004379562
6     Kidney      5  30 676  9  7        37 0.7780865 0.8108108 0.013138686
7   Pancreas     10  21 690  7  4        25 0.7859288 0.8400000 0.010043042
8    Adipose      0  53 651  9  9        62 0.8412023 0.8548387 0.013636364
9        Eye      2  12 697  6  7        19 0.6396622 0.6315789 0.008534851
10    Muscle      9  20 682 10 10        30 0.6522158 0.6666667 0.014450867
11       MOE      8  10 711  1  0        10 0.9527928 1.0000000 0.001404494
12    Testes     12  11 707  2  2        13 0.8433330 0.8461538 0.002820874
13      Skin

In [8]:
%%R
str(resnet_data)

'data.frame':	722 obs. of  17 variables:
 $ sample_ids         : chr  "SRR1810239" "SRR7161462" "SRR2033443" "SRR6680502" ...
 $ tissues            : chr  "Liver" "Intestine" "Liver" "Liver" ...
 $ target             : int  6 4 6 6 3 4 1 6 6 7 ...
 $ prediction         : int  6 4 0 6 4 4 1 6 6 7 ...
 $ confidence_class_0 : num  9.31e-07 8.67e-07 1.00 2.20e-09 2.07e-05 ...
 $ confidence_class_1 : num  4.78e-03 2.46e-05 5.38e-05 1.56e-06 1.16e-01 ...
 $ confidence_class_2 : num  4.47e-08 2.35e-04 1.95e-11 9.92e-07 1.58e-04 ...
 $ confidence_class_3 : num  2.33e-07 1.29e-09 4.01e-05 6.59e-11 3.20e-02 ...
 $ confidence_class_4 : num  2.02e-02 9.98e-01 1.03e-05 6.55e-04 8.26e-01 ...
 $ confidence_class_5 : num  2.11e-04 1.95e-04 3.42e-06 8.52e-06 1.57e-03 ...
 $ confidence_class_6 : num  9.74e-01 1.46e-05 2.52e-07 9.95e-01 1.09e-02 ...
 $ confidence_class_7 : num  5.54e-05 8.03e-09 8.47e-05 9.04e-07 1.64e-06 ...
 $ confidence_class_8 : num  6.60e-05 8.12e-08 1.81e-05 8.79e-14 1.13e-02 ...
 

In [ ]:
%%R
unique_tissues <- unique(c(resnet_data$target, resnet_data$prediction))
names <- unique(resnet_data$tissues)
resnet_results <- data.frame(names = names, tissue = unique_tissues)

resnet_results$TP <- 0
resnet_results$TN <- 0
resnet_results$FP <- 0
resnet_results$FN <- 0

for (tissue in unique_tissues) {
  for (i in 1:nrow(resnet_data)) {
    if (resnet_data$target[i] == tissue) {
      if (resnet_data$prediction[i] == tissue) {
        resnet_results[resnet_results$tissue == tissue, "TP"] <- resnet_results[resnet_results$tissue == tissue, "TP"] + 1
      } else {
        resnet_results[resnet_results$tissue == tissue, "FN"] <- resnet_results[resnet_results$tissue == tissue, "FN"] + 1
      }
    } else {
      if (resnet_data$prediction[i] == tissue) {
        resnet_results[resnet_results$tissue == tissue, "FP"] <- resnet_results[resnet_results$tissue == tissue, "FP"] + 1
      } else {
        resnet_results[resnet_results$tissue == tissue, "TN"] <- resnet_results[resnet_results$tissue == tissue, "TN"] + 1
      }
    }
  }
}

resnet_results$POSITIVES <- resnet_results$TP + resnet_results$FN

resnet_results$MCC <- (resnet_results$TP * resnet_results$TN - resnet_results$FP * resnet_results$FN) / 
  sqrt((resnet_results$TP + resnet_results$FP) * (resnet_results$TP + resnet_results$FN) * (resnet_results$TN + resnet_results$FP) * (resnet_results$TN + resnet_results$FN))

resnet_results$TPR <- resnet_results$TP / (resnet_results$TP + resnet_results$FN)

resnet_results$FPR <- resnet_results$FP / (resnet_results$FP + resnet_results$TN)

resnet_results$TNR <- resnet_results$TN / (resnet_results$TN + resnet_results$FP)

resnet_results$FNR <- resnet_results$FN / (resnet_results$FN + resnet_results$TP)

resnet_results$Precision <- resnet_results$TP / (resnet_results$TP + resnet_results$FP)

resnet_results$Recall <- resnet_results$TP / (resnet_results$TP + resnet_results$FN)

resnet_results$F1_score <- 2 * (resnet_results$Precision * resnet_results$Recall)/(resnet_results$Precision + resnet_results$Recall)

print(resnet_results)

write.table(resnet_results, file = paste(OUT_DIR, "resnet_MCC_Result.txt"), sep="\t", row.names = FALSE, quote = FALSE)


       names tissue  TP  TN FP FN POSITIVES       MCC       TPR         FPR
1      Liver      6 202 502  3 15       217 0.9405125 0.9308756 0.005940594
2  Intestine      4  56 651  7  8        64 0.8705335 0.8750000 0.010638298
3      Heart      3  16 694  8  4        20 0.7219534 0.8000000 0.011396011
4      Brain      1 168 537  7 10       178 0.9363200 0.9438202 0.012867647
5       Lung      7  36 683  2  1        37 0.9579038 0.9729730 0.002919708
6     Kidney      5  34 673 12  3        37 0.8137889 0.9189189 0.017518248
7   Pancreas     10  19 687 10  6        25 0.6943164 0.7600000 0.014347202
8    Adipose      0  49 654  6 13        62 0.8250939 0.7903226 0.009090909
9        Eye      2  17 703  0  2        19 0.9445626 0.8947368 0.000000000
10    Muscle      9  26 682 10  4        30 0.7813578 0.8666667 0.014450867
11       MOE      8  10 709  3  0        10 0.8752083 1.0000000 0.004213483
12    Testes     12  11 708  1  2        13 0.8786174 0.8461538 0.001410437
13      Skin

In [ ]:
%%R
ari_row_gnn <- data.frame(names = paste("gnn_ARI: ", gnn_ari), tissue = NA, TP = NA, TN = NA, FP = NA, FN = NA, POSITIVES = NA, MCC = NA, TPR = NA, FPR = NA, TNR = NA, FNR = NA, Precision = NA, Recall = NA, F1_score = NA)
ari_row_resnet <- data.frame(names = paste("resnet_ARI: ", resnet_ari), tissue = NA, TP = NA, TN = NA, FP = NA, FN = NA, POSITIVES = NA, MCC = NA, TPR = NA, FPR = NA, TNR = NA, FNR = NA, Precision = NA, Recall = NA, F1_score = NA)

gnn_results <- rbind(gnn_results, ari_row_gnn)
resnet_results <- rbind(resnet_results, ari_row_resnet)

if (all(names(gnn_results) == names(resnet_results))) {
  combined_df <- rbind(gnn_results, resnet_results)
  print(combined_df)
  write.table(combined_df, file = paste(OUT_DIR, "MCC_Result.txt"), sep="\t", row.names = FALSE, quote = FALSE)
} else {
  print("Column names or orders do not match between data frames.")
}


                            names tissue  TP  TN FP FN POSITIVES       MCC
1                           Liver      6 199 495 10 18       217 0.9071029
2                       Intestine      4  54 652  6 10        64 0.8593955
3                           Heart      3  12 691 11  8        20 0.5460590
4                           Brain      1 161 526 18 17       178 0.8697684
5                            Lung      7  35 682  3  2        37 0.9297737
6                          Kidney      5  30 676  9  7        37 0.7780865
7                        Pancreas     10  21 690  7  4        25 0.7859288
8                         Adipose      0  53 651  9  9        62 0.8412023
9                             Eye      2  12 697  6  7        19 0.6396622
10                         Muscle      9  20 682 10 10        30 0.6522158
11                            MOE      8  10 711  1  0        10 0.9527928
12                         Testes     12  11 707  2  2        13 0.8433330
13                       